# Calculate CMIP Indeces

This file 
1. calculates all the indeces for all months for all cesmFullForcingFiles.



In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils.timePeriod as tp
import utils.compound as compound


In [2]:
import xarray
import numpy
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 1. Calculate CESM Indeces

import warnings
warnings.filterwarnings('ignore')

For each experiment in historical, calculate indices

In [4]:
deckSet=['piControl','historical']
scenarioSet=['ssp126', 'ssp245', 'ssp370','ssp585']
experimentSet=[*deckSet, *scenarioSet]

In [5]:
#need to double check about the model run used!
modelSet=_model.scenarioMip
#modelSet=numpy.array([
    #['CSIRO-ARCCSS', 'ACCESS-CM2', 'r1i1p1f1', 'r1i1p1f1'],
    #['CSIRO', 'ACCESS-ESM1-5', 'r1i1p1f1', 'r1i1p1f1'],
    #['AWI', 'AWI-CM-1-1-MR', 'r1i1p1f1', 'r1i1p1f1'],
    #['BCC', 'BCC-CSM2-MR', 'r1i1p1f1', 'r1i1p1f1'],
    #['CAMS', 'CAMS-CSM1-0', 'r1i1p1f1', 'r1i1p1f1'],
    #['CAS', 'CAS-ESM2-0', 'r1i1p1f1', 'r1i1p1f1'],
    #['NCAR', 'CESM2', 'r1i1p1f1', 'r10i1p1f1'],
    #['NCAR', 'CESM2-WACCM', 'r1i1p1f1', 'r1i1p1f1'],
    #['THU', 'CIESM', 'r1i1p1f1', 'r1i1p1f1'],
    #['CMCC', 'CMCC-CM2-SR5', 'r1i1p1f1', 'r1i1p1f1'],
    #['CMCC', 'CMCC-ESM2', 'r1i1p1f1', 'r1i1p1f1'],
    #['CNRM-CERFACS', 'CNRM-CM6-1', 'r1i1p1f2', 'r1i1p1f2'],
    #['CNRM-CERFACS', 'CNRM-CM6-1-HR', 'r1i1p1f2', 'r1i1p1f2'],
    #['CNRM-CERFACS', 'CNRM-ESM2-1', 'r1i1p1f2', 'r1i1p1f2'],
    #['CCCma', 'CanESM5', 'r1i1p1f1', 'r1i1p1f1'],
    #['CCCma', 'CanESM5-CanOE', 'r1i1p2f1', 'r1i1p2f1'],
    #['E3SM-Project', 'E3SM-1-1', 'r1i1p1f1', 'r1i1p1f1'],
    #['EC-Earth-Consortium', 'EC-Earth3', 'r1i1p1f1', 'r1i1p1f1'],
    #['EC-Earth-Consortium', 'EC-Earth3-CC', 'r1i1p1f1', 'r1i1p1f1'],
    #['EC-Earth-Consortium', 'EC-Earth3-Veg', 'r1i1p1f1', 'r1i1p1f1'],
    #['EC-Earth-Consortium', 'EC-Earth3-Veg-LR', 'r1i1p1f1', 'r1i1p1f1'],
    #['FIO-QLNM', 'FIO-ESM-2-0', 'r1i1p1f1', 'r1i1p1f1'],
    #['NOAA-GFDL', 'GFDL-CM4', 'r1i1p1f1', 'r1i1p1f1'],
    #['NOAA-GFDL', 'GFDL-ESM4', 'r1i1p1f1', 'r1i1p1f1'], #where is piControl?
    #['NASA-GISS', 'GISS-E2-1-G', 'r1i1p1f2', 'r1i1p1f2'],
    #['MOHC', 'HadGEM3-GC31-LL', 'r1i1p1f1', 'r1i1p1f3'], #NB the piControl used a slightly older version of the forcings.
    #['MOHC', 'HadGEM3-GC31-MM', 'r1i1p1f1', 'r1i1p1f3'],
    #['CCCR-IITM', 'IITM-ESM', 'r1i1p1f1', 'r1i1p1f1'],
    #['INM', 'INM-CM4-8', 'r1i1p1f1', 'r1i1p1f1'],
    #['INM', 'INM-CM5-0', 'r1i1p1f1', 'r1i1p1f1'],
    #['IPSL', 'IPSL-CM6A-LR', 'r1i1p1f1', 'r1i1p1f1'],
    #['NIMS-KMA', 'KACE-1-0-G', 'r1i1p1f1', 'r1i1p1f1'],
    #['KIOST', 'KIOST-ESM', 'r1i1p1f1', 'r1i1p1f1'],
    #['UA', 'MCM-UA-1-0', 'r1i1p1f1', 'r1i1p1f2'],
    #['MIROC', 'MIROC-ES2L', 'r1i1p1f2', 'r1i1p1f2'],
    #['MIROC', 'MIROC6', 'r1i1p1f1', 'r1i1p1f1'],
    #['MPI-M', 'MPI-ESM1-2-HR', 'r1i1p1f1', 'r1i1p1f1'],
    #['MPI-M', 'MPI-ESM1-2-LR', 'r1i1p1f1', 'r1i1p1f2'],
    #['MRI', 'MRI-ESM2-0', 'r1i1p1f1', 'r1i1p1f1'],
    #['NUIST', 'NESM3', 'r1i1p1f1', 'r1i1p1f1'],
    #['NCC', 'NorESM2-LM', 'r1i1p1f1', 'r1i1p1f1'],
    #['NCC', 'NorESM2-MM', 'r1i1p1f1', 'r1i1p1f1'],
    #['MOHC', 'UKESM1-0-LL', 'r1i1p1f2', 'r1i1p1f2']
#])

##no SSP 585
    #['EC-Earth-Consortium', 'EC-Earth3-AerChem', 'r1i1p1f1', 'r1i1p1f1'],
    #['IPSL', 'IPSL-CM5A2-INCA', 'r1i1p1f1', 'r1i1p1f1'],
    #['HAMMOZ-Consortium', 'MPI-ESM-1-2-HAM', 'r1i1p1f1', 'r1i1p1f1'],

In [29]:
pslIndex

<xarray.DataArray 'sam' (time: 12000)>
dask.array<sub, shape=(12000,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 0101-01-16 00:00:00 ... 1100-12-16 00:00:00
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12

In [41]:
for iModel in modelSet:
    
    print(iModel)
    
    try: 
        print(iModel[1] + ' starting')
        #SST
        tsDs = fh.loadModelData(iModel[1], 'ts_Amon', 'piControl', iModel[2]).ts.to_dataset()
        controlDs=tsDs.assign_attrs({'project_id':'CMIP'})
        sstClimat=sst.calculateClimatology(controlDs)

        pslControlDs=fh.loadModelData(iModel[1], 'psl_Amon', 'piControl', iModel[2])
        pslClimat=psl.calculateClimatology(pslControlDs)

        sstIndex = sst.calculateIndex(controlDs, sstClimat) #(reducing the model set at this step could save time?)
        pslIndex, junk = psl.calculateSamIndex(pslControlDs, pslClimat)
        indeces = xarray.merge([pslIndex, sstIndex])
        indeces.assign_attrs(climatology='full length of pi Control')
        print('Caclulating control ...')
        tp.averageForTimePeriod(indeces).to_netcdf(
            'results/cmipWarmSeasonIndeces/'+iModel[1]+'_piControl.nc')

        #historical
        tsDs = fh.loadModelData(iModel[1], 'ts_Amon', 'historical', iModel[3]).ts.to_dataset()
        #sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
        sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

        sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)

        pslDs = fh.loadModelData(iModel[1], 'psl_Amon', 'historical',iModel[3])
        pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)
        indeces = xarray.merge([pslIndex, sstIndex])
        indeces.assign_attrs(climatology='full length of pi Control')
        #print(indeces)
        print('Caclulating historical ...')
        historicalIndeces=indeces.copy()
        historicalIndeces.load()
        tp.averageForTimePeriod(historicalIndeces).to_netcdf('results/cmipWarmSeasonIndeces/'+iModel[1]+'_historical.nc')
        
        for experiment in scenarioSet: 
            try:
                variant = iModel[3]
                
                tsDs = fh.loadModelData(iModel[1], 'ts_Amon', experiment, variant).ts.to_dataset()
                #sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
                sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

                sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)

                pslDs = fh.loadModelData(iModel[1], 'psl_Amon', experiment,variant)
                pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)

                indeces = xarray.concat([
                    historicalIndeces, 
                                         xarray.merge([pslIndex, sstIndex])
                    ], 'time')

                indeces.assign_attrs(climatology='full length of pi Control')
                #print(indeces)
                print('Caclulating ...')
                tp.averageForTimePeriod(indeces).to_netcdf('results/cmipWarmSeasonIndeces/'+iModel[1]+'_'+experiment + '.nc')

            except Exception as e:
                print(iModel[1] + experiment + " not completed: ")
                print(e)
                
            else:
                print(iModel[1] + experiment + ' complete')  

    except Exception as e:
        print(iModel[1] + "Climatology did not calculate")
        print(e)
        
    else:
        print(iModel[1] + ' finished')


['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1' 'r1i1p1f1']
ACCESS-CM2 starting
Caclulating control ...
Caclulating historical ...
Caclulating ...
ACCESS-CM2ssp126 complete
Caclulating ...
ACCESS-CM2ssp245 complete
Caclulating ...
ACCESS-CM2ssp370 complete
Caclulating ...
ACCESS-CM2ssp585 complete
ACCESS-CM2 finished
['CSIRO' 'ACCESS-ESM1-5' 'r1i1p1f1' 'r1i1p1f1']
ACCESS-ESM1-5 starting
Caclulating control ...
Caclulating historical ...


KeyboardInterrupt: 

This grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

Theres a few examples of output at the bottom to sanity check this is meaningful

The indeces for each month have already been calculated. So loading up the saved versions, and putting them into a single xarray.

In [ ]:
_model.scenarioMip[:,1]

In [ ]:
for experiment in experimentSet: 
    print(experiment)
    timePIndeces=list()

    for model in _model.scenarioMip[:,1]:
        try:
            indecesDs = xarray.open_dataset('results/cmipWarmSeasonIndeces/' + model +'_'+ experiment + '.nc')
            indecesDs['model']=model
            timePIndeces.append(indecesDs)
        except Exception as e:
            print(e) 
    results=xarray.concat(timePIndeces, 'model')

    results=results.assign_attrs({'experiment':experiment,**indecesDs.attrs,**_index.monthsOfInterest})

    results.to_netcdf('results/cmip6'+experiment+'TradIndeces.nc')

In [ ]:
results.sel(year=slice(2015,2099))

Some sanity checks to see if it looks ok:


In [ ]:
results=xarray.open_dataset('results/cmip6historicalTradIndeces.nc')

In [ ]:
str(results.isel(model=6).model.values)

In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

results.isel(model=5).nino34.plot(label='nino34')
results.isel(model=5).nino34NoDetrend.plot(label='No Detrend')

#Tidy up a bit
plt.title(str(results.isel(model=5).model.values) + " Historical Nino34")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-model mean?

mean = results.sam.mean(dim='model')
std = results.sam.std(dim='model')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.mean(dim='model')
std = results.std(dim='model')

In [ ]:
std

# 2. Calculate Compound Years for CMIP

# Define a criteria to determine events

So we are going to define events as follows:
- For SST indeces (aka Enso and Iod), this is 1 sigma above/below mean
- For PSL indeces (aka Sam) this is 0.5 sigma above/below mean

Using a piControl climatology

In [ ]:
piControlXr=xarray.open_dataset('results/cmip6piControlTradIndeces.nc')

piControlXr.model.values

In [ ]:
# This should calculate stds for all experiments and all indeces
stdXr = piControlXr.std(dim='year')

# define some criteria that we want to use to identifiy it thats an event
criteriaXr= xarray.merge(
    (stdXr[['nino34','dmi']], stdXr['sam']/2)
)

In [ ]:
criteriaXr

# Calculate the compounding years and plot an example

In [ ]:
for experiment in scenarioSet:

    # Here is the data
    indecesXr = xarray.open_dataset('results/cmip6'+experiment+'TradIndeces.nc')

    firePos, fireNeg = compound.applyCriteria(indecesXr, criteriaXr)

    firePos=compound.compound(firePos)

    fireNeg=compound.compound(fireNeg)

    firePos.to_netcdf('results/cmip6'+experiment+'FirePos.nc')

    fireNeg.to_netcdf('results/cmip6'+experiment+'FireNeg.nc')

In [ ]:
indecesXr.sel(year=2014)

In [ ]:
firePos

In [ ]:
modToPlot = 'UKESM1-0-LL'

toPlot=3*firePos.all3.sel(model=modToPlot)

plt.figure(figsize=(12,12))

plt.vlines(
    x=firePos.year, 
    ymin=0,
    ymax=toPlot,
    label='all3'
)

colors=['orange', 'purple', 'green']
iCol = 0

for iPair in firePos.attrs['pairs']:
    plt.vlines(
        x=firePos.year, 
        ymin=0,
        ymax=2*firePos[iPair].sel(model=modToPlot),
        color=colors[iCol],
        label=iPair
    )
    iCol+=1
plt.legend()
plt.xlim(1850,2100)
plt.title(modToPlot+': Pairs and all-three compounds for individal years')